<a href="https://colab.research.google.com/github/alcobeard/otus/blob/master/mnist_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install jupyterthemes

In [0]:
from jupyterthemes import jtplot

jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [0]:
# !cp "/content/drive/My Drive/Colab Notebooks/DL Otus/Homework/2/utils.py" .

In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DL Otus/Homework/2/')

In [0]:
import os
import subprocess
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

from utils import mnist

In [0]:
# Если доступна видеокрта - будем считаться на ней
# Если нет - на ЦПУ
# Обязательно используем .to(device) на torch.tensor()

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(subprocess.getoutput("nvidia-smi"))
    print()
    print(f"GPUs in  system:        {torch.cuda.device_count()}")
    print(f"GPU compute capability: {torch.cuda.get_device_capability(device)}")
    print(f"GPU name:               {torch.cuda.get_device_name()}")
else:
    device = torch.device('cpu')
device

In [0]:
torch.cuda.empty_cache()

In [0]:
# Описываем класс, создающий сеть
class Net(nn.Module):
    def __init__(self, log_softmax=False):
        super(Net, self).__init__()
        inputs=28*28
        l1_hidden_neurons=5
        outputs=10
        self.fc1 = nn.Linear(inputs, l1_hidden_neurons)
        self.act1 = torch.nn.Sigmoid() # не используется в forward
        self.fc2 = nn.Linear(l1_hidden_neurons, 128)
        self.fc3 = nn.Linear(128, outputs)
        self.log_softmax = log_softmax
        self.optim = optim.SGD(self.parameters(), lr=0.1)
        
    def forward(self, x):
        x = x.view(-1, 28*28) # выпрямляем в вектор размерности 1 на 28х28
        
        x = torch.sigmoid(self.fc1(x)) # активируем функцию через сигмоид(суммируем данные первым слоем)
        x = torch.sigmoid(self.fc2(x)) # активируем функцию через сигмоид(суммируем данные вторым слоем)
        
        x = self.fc3(x) # суммируем выходы второго слоя
        
        # и в зависимости от запрошенного, используем разные softmax'ы
        if self.log_softmax:
            x = F.log_softmax(x, dim=1) # log_softmax
        else:
            x = torch.log(F.softmax(x, dim=1)) # log от softmax
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [0]:
def train(models, epoch, train_stats):
    # читаем батчами по 50 экземпляров каждый (настройки dataset_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        # данные перемещаем на нужное устройство (CPU/GPU)
        data = data.to(device)
        target = target.to(device)
        
        for model in models:
            # обнулили накопленные градиенты
            model.optim.zero_grad()
            # посчитали ответы сети с текущими весами
            output = model.forward(data)
            # посчитали ошибку
            loss = model.loss(output, target)
            # посчитали градиент
            loss.backward()
            # и проапдейтили веса новыми значениями 
            model.optim.step()
        
        # раз в (batch_size * 200) == 10000 выведем статистику  
        if batch_idx % 200 == 0:
            epoch_stats = f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({round(100. * batch_idx / len(train_loader), 2)}%)]\t\t"
            loss_stats = f"Losses: "
            for idx, model in enumerate(models):
                train_stats[idx] += [model._loss.item()]
                loss_stats += f" {idx}: {round(model._loss.item(),4)}\t"
            print(epoch_stats + loss_stats)
            
    # перед завершением функции покажем посленюю статистику, т.к. она не попала  в цикл        
    batch_idx += 1 # потому что индекс с нуля
    epoch_stats = f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({round(100. * batch_idx / len(train_loader), 2)}%)]\t\t"
    loss_stats = f"Losses: "
    for idx, model in enumerate(models):
        train_stats[idx] += [model._loss.item()]
        loss_stats += f" {idx}: {round(model._loss.item(),4)}\t"
    print(epoch_stats + loss_stats)

In [0]:
def test(models, model_test_loss, model_test_accuracy):
    # будем считать статистики по каждой модели отдельно
    test_loss = [0]*len(models) # сюда накапливать величину лосса
    correct = [0]*len(models) # а сюда плюсовать правильные ответы
    with torch.no_grad():
        for data, target in test_loader:
            # тестовые данные закидываем на CPU\GPU:
            data = data.to(device)
            target = target.to(device)
            
            # смотрим какие ответы нам дает каждая модель на тестовой выборке:
            output = []
            for model in models:
                output += [model.forward(data)]

            # теперь подсчтаем статистики для каждой модели
            for i, model in enumerate(models):
                # запишем сумму ошибок каждой модели
                test_loss[i] += model.loss(output[i], target, reduction='sum').item()
                # запишем индекс самого вероятного класса каждой модели
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                # плюсик в правильные ответы модели если ответ совпал с правильным
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    # посчитали сумму лосса и количество правильных ответов каждой модели
    for i in range(len(models)):
        # теперь усредним лосс каждой модели
        test_loss[i] /= len(test_loader.dataset)
    # сумму правильных ответов каждой модели делим на воличество примеров, получаем процент правильных ответов каждой модели в массив
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    # и вывод результата подсчета
    # 0: Loss: 0.1010	Accuracy: 9679/10000 (97%)
    lines =""
    for i, model in enumerate(models):
        model_test_loss[i] += [test_loss[i]]
        model_test_accuracy[i] += [correct[i].item()/len(test_loader.dataset)]
        lines += f"Model #{i}\t Loss: {round(test_loss[i],4)}\t "
        lines += f"Accuracy:{correct[i]}/{len(test_loader.dataset)} "
        lines += f"({round(correct_pct[i].item(),2)}%)\n"
    report = 'Test set:\n' + lines
    
    print(report)

In [0]:
test_loader, train_loader = mnist()

In [0]:
%%time
# Достаточно одной модели с log_softmax
models = [Net(True).to(device)]

epoch_count = 500

# будем накапливать статистику для построения графиков
model_train_loss = []
model_test_loss = []
model_test_accuracy = []
# зададим массив так, чтобы первым элементом накапливался массив по первой модели, вторым по второй и т.д.
for i, model in enumerate(models):
    model_train_loss += [[]]
    model_test_loss += [[]]
    model_test_accuracy += [[]]

# стартуем обучение сетей
for epoch in range(1, epoch_count+1):
    train(models, epoch, model_train_loss)
    test(models, model_test_loss, model_test_accuracy)

In [0]:
for i, model in enumerate(models):
    model_train_loss[i].pop(0)
    normalized_test_loss = []
    normalized_test_accuracy = []
    for j in model_test_accuracy[i]:
        normalized_test_accuracy.extend([j]*2)
    for j in model_test_loss[i]:
        normalized_test_loss.extend([j]*2)
    normalized_test_accuracy.pop(0)
    normalized_test_loss.pop(0)

    plt.figure(figsize=(10,5))
    plt.title(f"Model #{i} loss")
    plt.plot(model_train_loss[i], linestyle="-", color=[.153,.255,.255,1], label="train")
    plt.plot(normalized_test_loss, linestyle="-", color=[1, .1, .1, 1.0], label="test")
    plt.plot(normalized_test_accuracy, linestyle="-", color=[0.1, 1., .1, 0.5], label="accuracy")
    plt.legend(loc="upper right")
    plt.grid()
    plt.show()